<h1><center>Lab 2: Classification</center></h1>
<h3><center>A Deeper Analysis of Covid-19 Data</center></h3>
<p><center>DS 7331</center></p>
<p><center>Created by Sadik Aman, Dawn Bowerman, Zachary Harris, Alexandre Jasserme</center></p>

<p><center>Sections of this code was adapted from: 
    <li>https://github.com/jakemdrew/DataMiningNotebooks</li>

In [1]:
from __future__ import print_function

#Libraries
import plotly
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.pyplot import figure
from sklearn.model_selection import TimeSeriesSplit
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics as mt
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit

df = pd.read_csv('Data/owid-covid-data_modified.csv') # read in the csv file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34965 entries, 0 to 34964
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   continent                       34965 non-null  object 
 1   location                        34965 non-null  object 
 2   date                            34965 non-null  object 
 3   new_cases                       34965 non-null  float64
 4   new_cases_smoothed              34965 non-null  float64
 5   new_deaths                      34965 non-null  float64
 6   new_deaths_smoothed             34965 non-null  float64
 7   reproduction_rate               34965 non-null  float64
 8   new_vaccinations_smoothed       34965 non-null  float64
 9   new_people_vaccinated_smoothed  34965 non-null  float64
 10  stringency_index                34965 non-null  float64
 11  population                      34965 non-null  int64  
 12  population_density              

## Data Preparation 

> [10 points] Define and prepare your class variables. Use proper variable representations
(int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality
reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

In [2]:
# Ideas from https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv

# Sorting data frame by date column
df['date'] = pd.to_datetime(df['date']) # Converting data columnn to datetime

df = df.sort_values(by='date', ascending=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   continent                       34965 non-null  object        
 1   location                        34965 non-null  object        
 2   date                            34965 non-null  datetime64[ns]
 3   new_cases                       34965 non-null  float64       
 4   new_cases_smoothed              34965 non-null  float64       
 5   new_deaths                      34965 non-null  float64       
 6   new_deaths_smoothed             34965 non-null  float64       
 7   reproduction_rate               34965 non-null  float64       
 8   new_vaccinations_smoothed       34965 non-null  float64       
 9   new_people_vaccinated_smoothed  34965 non-null  float64       
 10  stringency_index                34965 non-null  float64       
 11  po

In [3]:
covid_df = df.drop(["new_deaths_smoothed",
                    "new_cases",
                    "new_cases_smoothed",
                    "continent",
                    "location",
                    "stringency_range",
                    "new_cases_range",
                    "date",
                    "new_people_vaccinated_smoothed",
                    "population_density",
                    "aged_70_older",
                    "cardiovasc_death_rate",
                    "new_deaths_range"], axis=1)

covid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34965 entries, 0 to 34964
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   new_deaths                  34965 non-null  float64
 1   reproduction_rate           34965 non-null  float64
 2   new_vaccinations_smoothed   34965 non-null  float64
 3   stringency_index            34965 non-null  float64
 4   population                  34965 non-null  int64  
 5   median_age                  34965 non-null  float64
 6   aged_65_older               34965 non-null  float64
 7   gdp_per_capita              34965 non-null  float64
 8   diabetes_prevalence         34965 non-null  float64
 9   handwashing_facilities      34965 non-null  float64
 10  hospital_beds_per_thousand  34965 non-null  float64
 11  life_expectancy             34965 non-null  float64
 12  human_development_index     34965 non-null  float64
dtypes: float64(12), int64(1)
memory

In [4]:
print ('Size of the dataset:', covid_df.shape)
print ('Number of unique classes:', len(covid_df.new_deaths))

Size of the dataset: (34965, 13)
Number of unique classes: 34965


In [5]:
## KFold cross-validation

#separate the predicting attribute into Y for model training
covid_target = covid_df.new_deaths

#separate the other attributes from the predicting attribute
covid_df = covid_df.drop("new_deaths",axis=1)

## Define variables for the for loop
kf = KFold(n_splits=10)
RMSE_sum=0
RMSE_length=10
X = np.array(covid_df)
y = np.array(covid_target)
print(X)
print(y)
print(covid_target)

[[9.80000000e-01 1.41150000e+04 4.63000000e+01 ... 5.00000000e-01
  6.48300000e+01 5.11000000e-01]
 [3.23548387e-01 4.30000000e+02 3.79600000e+01 ... 5.06451613e-01
  8.25900000e+01 5.45586022e-01]
 [1.16000000e+00 8.21700000e+03 5.18500000e+01 ... 6.00000000e-01
  7.43000000e+01 6.63000000e-01]
 ...
 [1.18000000e+00 6.46436667e+04 4.24684211e+01 ... 2.10000000e+00
  7.71500000e+01 7.77000000e-01]
 [2.23000000e+00 7.89200000e+03 3.34866667e+01 ... 1.20000000e+00
  8.02300000e+01 8.48000000e-01]
 [8.00000000e-01 8.78800000e+03 5.27800000e+01 ... 1.70000000e+00
  6.14900000e+01 5.71000000e-01]]
[91.  0. 67. ... 10.  0. 13.]
0         91.0
3515       0.0
13690     67.0
21275    201.0
31265      0.0
         ...  
14429      0.0
3144       0.0
31819     10.0
26269      0.0
34964     13.0
Name: new_deaths, Length: 34965, dtype: float64


In [6]:
  for loop_number, (train, test) in enumerate(kf.split(X)):
    
    
    ## Get Training Matrix and Vector

    training_X_array = X[train]
    training_y_array = y[train]

    ## Get Testing Matrix Values

    X_test_array = X[test]
    y_actual_values = y[test]

    ## Fit the Linear Regression Model

    lr_model = LinearRegression().fit(training_X_array, training_y_array)

    ## Compute the predictions for the test data

    prediction = lr_model.predict(X_test_array)      
    deaths_probabilites = np.array(prediction)   

    ## Calculate the RMSE

    RMSE_cross_fold = mean_squared_error(deaths_probabilites, y_actual_values)

    ## Add each RMSE_cross_fold value to the sum

    RMSE_sum=RMSE_cross_fold+RMSE_sum

## Calculate the average and print    

RMSE_cross_fold_avg=RMSE_sum/RMSE_length

print('The Mean RMSE across all folds is',RMSE_cross_fold_avg)

The Mean RMSE across all folds is 24996.518383436614
